In [1]:
import requests
from bs4 import BeautifulSoup , SoupStrainer
import traceback
import pandas as pd
import calendar
import datetime
import time
import winsound
import os
import re
from googleapiclient import discovery
import json
#pip install -U oauth2client
from oauth2client.client import GoogleCredentials
from google.oauth2 import service_account
from datetime import date
#pip install pandas-gbq -U
from google.cloud import bigquery
from google.cloud import secretmanager
from google.cloud import resourcemanager_v3
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
from email.mime.application import MIMEApplication
import re
import io

In [2]:
def send_email_with_attachment(
    receiver_emails: list,
    subject: str,
    body: str,
    filename: str,
    dataframe: pd.DataFrame()
):
    #Create IO buffer to add dataframe
    s_buf = io.StringIO() 
    dataframe.to_csv(s_buf)
    s_buf
    #Mail server details
    smtp_server = "smtp.gmail.com"
    sender_email = 'reporting.everydaymarket@woolworths.com.au'
    port = 587  # TLS port
    password = "dsjexgugwkyyptuy"
    #Create mail
    message = MIMEMultipart()
    message['From'] = sender_email
    message['To'] = ', '.join(email)
    message['Subject'] = subject
    # Add body to email
    message.attach(MIMEText(body, 'plain'))
    attachment = MIMEApplication(s_buf.getvalue())  ### Here is where the df is attached
    attachment['Content-Disposition'] = 'attachment; filename="{}"'.format(filename)
    message.attach(attachment)



    # Send email
    text = message.as_string()
    with smtplib.SMTP(smtp_server, port) as server:
            server.starttls()  # Secure the connection
            server.login(sender_email, password)
            server.sendmail(sender_email, email, text)
    return "Success"

In [3]:
#Get data from BQ
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.getcwd() + "\\application_default_credentials.json"
credentials = GoogleCredentials.get_application_default()
sm_client = secretmanager.SecretManagerServiceClient()
sa_key = json.loads(sm_client.access_secret_version(request={"name":'projects/59083446455/secrets/ed-mktplc-dev/versions/1'}).payload.data.decode("UTF-8"))

credentials = service_account.Credentials.from_service_account_info(sa_key)
#Setup Application Default Credentials -- https://cloud.google.com/docs/authentication/client-libraries#python
client = bigquery.Client(project='gcp-wow-rwds-ai-ed-mktplc-dev', credentials=credentials)

#Details for MyDeal Delivery
email = ['ygupta@woolworths.com.au','gwalne@woolworths.com.au']
query_text ="""select 

traderOrderID,
mpOrderID,
invoiceID,
orderReference,
orderDateAEST,
mpOrderDate,
sellerName,
productName,
variantName,
MPArticleNumber,
quantityOrdered,
shopperId,
quantityShipped,
trackingNumber,
trackingLink,
promisedDeliveryDateMin,
promisedDeliveryDateMax,
dispatchDate,
dispatchTime,
shippingStatus,
carrier,
postcode,
lateDispatch,
potentialSplitShipping

 from 
gcp-wow-rwds-ai-ed-mktplc-dev.bi_reporting.vw_MarketPlace_Report_ShippingExperience
where sellerName = 'MyDeal'
and shippingStatus = 'Not Shipped';
                        """
subject = 'Non-Dispatch Shipping Data - MyDeal ' + date.today().strftime("%d/%m/%Y")
body = 'Please find attached the CSV export'
df_result_set = client.query(query_text).result().to_dataframe()
filename = 'non_dispatch_mydeal_' + date.today().strftime("%d/%m/%Y") + '.csv'
email_result = send_email_with_attachment(email,subject,body,filename, df_result_set)
print(email_result)

C:\Users\Yash Gupta\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\auth\_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Success
